## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-16-52-48 +0000,bbc,Investigation into bans for Grenfell firms paused,https://www.bbc.com/news/articles/cq537ey5ggjo
1,2025-07-10-16-43-05 +0000,bbc,UK faces rising and unpredictable threat from ...,https://www.bbc.com/news/articles/c98wjzj4jlpo
2,2025-07-10-16-40-36 +0000,nyt,"Israel Will Allow More Aid Into Gaza, Official...",https://www.nytimes.com/2025/07/10/world/middl...
3,2025-07-10-16-31-28 +0000,nyt,U.K. and France Agree to First-Ever Nuclear We...,https://www.nytimes.com/2025/07/10/world/europ...
4,2025-07-10-16-27-08 +0000,nyt,"England’s Sycamore Gap Tree, Felled by Vandals...",https://www.nytimes.com/2025/07/10/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2300/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
56,trump,17
86,russia,5
14,will,4
88,ukraine,4
99,new,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,39
46,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,38
13,2025-07-10-14-42-38 +0000,nyt,Trump Seems to Be Warming to What Europe Wants...,https://www.nytimes.com/2025/07/10/world/europ...,37
54,2025-07-09-21-39-44 +0000,nyt,Russia Makes Record Attack on Ukraine as Trump...,https://www.nytimes.com/2025/07/09/world/europ...,34
23,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,34


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,39,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
13,20,2025-07-10-14-42-38 +0000,nyt,Trump Seems to Be Warming to What Europe Wants...,https://www.nytimes.com/2025/07/10/world/europ...
46,17,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...
58,15,2025-07-09-20-25-00 +0000,wsj,Minutes from the Fed’s June policy meeting tea...,https://www.wsj.com/economy/central-banking/ju...
1,13,2025-07-10-16-43-05 +0000,bbc,UK faces rising and unpredictable threat from ...,https://www.bbc.com/news/articles/c98wjzj4jlpo
23,13,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
57,13,2025-07-09-20-36-58 +0000,bbc,'Huge wall of water': Texas man stood on meter...,https://www.bbc.com/news/articles/c3358pz41d6o
52,13,2025-07-09-22-33-33 +0000,nyt,"Not Invited to Dior’s Show in Paris, a Fashion...",https://www.nytimes.com/2025/07/09/style/dior-...
18,12,2025-07-10-13-24-21 +0000,nyt,A Lethal Israeli Airstrike Hits Near a Gaza Ai...,https://www.nytimes.com/2025/07/10/world/middl...
40,12,2025-07-10-05-25-38 +0000,bbc,Reform leader says council first to scrap net ...,https://www.bbc.com/news/articles/c3vde31lwpdo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
